In [1]:
!pip install nlpaug
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 7.3 MB/s eta 0:00:00a 0:00:01


In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from sklearn.metrics import f1_score, confusion_matrix, balanced_accuracy_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from torch.optim import AdamW, lr_scheduler
import nlpaug.augmenter.word as naw
import optuna
import shutil
import zipfile

In [ ]:
# Ensure output directory exists
output_dir = "/kaggle/working/output"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Load and prepare data
# original
df = pd.read_parquet("/kaggle/input/train-parquet")
df['label_int'] = df['label'].str.split("_").str[0].astype('int')

# augmented 
train1 = pd.read_csv('/kaggle/input/balanced/train1.csv')
train2 = pd.read_csv('/kaggle/input/balanced/train2.csv')
train3 = pd.read_csv('/kaggle/input/balanced/train3.csv')
train4 = pd.read_csv('/kaggle/input/balanced/train4.csv')

In [ ]:
# Split data
# train123 is the augmentated data (the train set that; splitted with random state 42)
texts = df["quote"].to_list()
labels = df["label_int"].to_list()

X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42, stratify=labels)

datasets = [train1, train2, train3, train4]

# Extract quotes and labels using list comprehension
texts = [ds['quote'] for ds in datasets]
labels = [ds['numeric_label'] for ds in datasets]

# Concatenate all texts and labels using pandas.concat
train1234_texts = pd.concat(texts, ignore_index=True)
train1234_labels = pd.concat(labels, ignore_index=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Create dictionary for label names
label_dict = df[['label_int', 'label']].drop_duplicates().set_index('label_int')['label'].to_dict()

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', do_lower_case=True)

# Dataset and DataLoader preparation
class QuotesDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

def encode_data(tokenizer, texts, labels, max_length):
    try:
        if isinstance(texts, pd.Series):
            texts = texts.tolist()
        if isinstance(labels, pd.Series):
            labels = labels.tolist()
            
        encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
        return QuotesDataset(encodings, labels)

    except Exception as e:
        print(f"Error during tokenization: {e}")
        return None

In [ ]:
# Training params
MAX_LENGTH = 365
TRAIN_BATCH_SIZE = 16
VAL_BATCH_SIZE = 64
LEARNING_RATE = 1e-5
STEP_SIZE = 2
GAMMA = 0.1
EPOCHS = 2

In [ ]:
texts = df["quote"].to_list()
labels = df["label_int"].to_list()

X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42, stratify=labels)

In [ ]:
train_dataset = encode_data(tokenizer, X_train, y_train, MAX_LENGTH)
train1234_dataset = encode_data(tokenizer, train1234_texts, train1234_labels, MAX_LENGTH)
val_dataset = encode_data(tokenizer, X_test, y_test, MAX_LENGTH)

train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
train1234_loader = DataLoader(train1234_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=VAL_BATCH_SIZE, shuffle=False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=8)
model.to(device)
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

In [ ]:
# Plotting function for accuracy
def plot_accuracies(training_accuracies, validation_accuracies):
    plt.figure(figsize=(8, 6))
    plt.plot(training_accuracies, label='Training Accuracy')
    plt.plot(validation_accuracies, label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig(f"{output_dir}/accuracy_plot.png")
    plt.close()

def plot_confusion_matrix(cm, class_labels, epoch, output_dir):
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
    plt.title(f'Confusion Matrix for Epoch {epoch + 1}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(os.path.join(output_dir, f'confusion_matrix_epoch_{epoch + 1}.png'))
    plt.close()


In [ ]:
# Training and validation loop
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []
metrics_df = pd.DataFrame()

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    correct_train = 0
    total_train = 0

    for batch in train_loader:
        optimizer.zero_grad()
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        predictions = torch.argmax(outputs.logits, dim=-1)
        correct_train += (predictions == batch['labels']).sum().item()
        total_train += batch['labels'].size(0)

    scheduler.step()
    train_losses.append(train_loss / len(train_loader))
    train_accuracies.append(correct_train / total_train)

    model.eval()
    val_loss = 0
    correct_val = 0
    total_val = 0
    all_predictions, all_true_labels = [], []

    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            val_loss += outputs.loss.item()
            predictions = torch.argmax(outputs.logits, dim=-1)
            all_predictions.extend(predictions.cpu().numpy())
            all_true_labels.extend(batch['labels'].cpu().numpy())

            correct_val += (predictions == batch['labels']).sum().item()
            total_val += batch['labels'].size(0)

    val_losses.append(val_loss / len(val_loader))
    val_accuracies.append(correct_val / total_val)

    # Compute confusion matrix
    cm = confusion_matrix(all_true_labels, all_predictions)
    class_labels = [label_dict.get(i, f'Class {i}') for i in range(len(np.unique(all_true_labels)))]
    plot_confusion_matrix(cm, class_labels, epoch, output_dir)

    # Calculate metrics
    balanced_acc = balanced_accuracy_score(all_true_labels, all_predictions)
    average_f1 = f1_score(all_true_labels, all_predictions, average='macro')
    weighted_f1 = f1_score(all_true_labels, all_predictions, average='weighted')
    f1_scores_per_class = f1_score(all_true_labels, all_predictions, average=None)
    precision_per_class = precision_score(all_true_labels, all_predictions, average=None, zero_division=0)
    recall_per_class = recall_score(all_true_labels, all_predictions, average=None, zero_division=0)

    # Append metrics to DataFrame with class labels
    epoch_metrics = {
        "Epoch": epoch + 1,
        "Train Loss": train_losses[-1],
        "Validation Loss": val_losses[-1],
        "Train Accuracy": train_accuracies[-1],
        "Validation Accuracy": val_accuracies[-1],
        "Balanced Accuracy": balanced_acc,
        "Average F1": average_f1,
        "Weighted F1": weighted_f1
    }
    epoch_metrics.update({f"{label_dict[i]} F1": f1_scores_per_class[i] for i in range(len(f1_scores_per_class))})
    epoch_metrics.update({f"{label_dict[i]} Precision": precision_per_class[i] for i in range(len(precision_per_class))})
    epoch_metrics.update({f"{label_dict[i]} Recall": recall_per_class[i] for i in range(len(recall_per_class))})

    metrics_df = pd.concat([metrics_df, pd.DataFrame([epoch_metrics])], ignore_index=True)

# Save metrics to CSV and plot accuracies
metrics_df.to_csv(f"{output_dir}/training_metrics.csv", index=False)
plot_accuracies(train_accuracies, val_accuracies)

# Print overall model accuracy
overall_accuracy = accuracy_score(y_test, all_predictions)
print(f"Overall Model Accuracy: {overall_accuracy:.4f}")

In [ ]:
def zip_directory(folder_path, output_path):
    """Zip the contents of an entire directory and save the archive to the specified output path."""
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Create a relative path for files to preserve the directory structure
                zipf.write(os.path.join(root, file),
                           os.path.relpath(os.path.join(root, file),
                                           os.path.join(folder_path, '..')))

In [ ]:
# Directory to be zipped
input_dir = '/kaggle/working/output'

# Output path for the zip file
zip_file_path = '/kaggle/working/output.zip'

# Creating the ZIP file
zip_directory(input_dir, zip_file_path)

print(f"Created zip file at: {zip_file_path}")

In [ ]:
def modify_model(model, num_trainable_layers, dropout_rate):
    # Freeze layers: Only the last 'num_trainable_layers' are trainable
    for name, param in model.named_parameters():
        if 'transformer.layer' in name:
            layer_num = int(name.split('.')[3])
            if layer_num < 6 - num_trainable_layers:
                param.requires_grad = False

    # Adjust dropout rates in transformer layers
    for layer in model.distilbert.transformer.layer:
        layer.attention.dropout.p = dropout_rate
        layer.sa_layer_norm.dropout.p = dropout_rate
        layer.ffn.dropout.p = dropout_rate
        layer.output_layer_norm.dropout.p = dropout_rate

    return model

In [ ]:
from transformers import DistilBertConfig

In [ ]:
def train_one_epoch(model, train_loader, optimizer, device):
    model.train()
    train_loss = 0
    correct_train = 0
    total_train = 0
    for batch in train_loader:
        optimizer.zero_grad()
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        predictions = torch.argmax(outputs.logits, dim=-1)
        correct_train += (predictions == batch['labels']).sum().item()
        total_train += batch['labels'].size(0)
    average_loss = train_loss / len(train_loader)
    accuracy = correct_train / total_train
    return average_loss, accuracy

def validate_model(model, val_loader, device):
    model.eval()
    val_loss = 0
    correct_val = 0
    total_val = 0
    all_predictions = []
    all_true_labels = []
    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            val_loss += outputs.loss.item()
            predictions = torch.argmax(outputs.logits, dim=-1)
            all_predictions.extend(predictions.cpu().numpy())
            all_true_labels.extend(batch['labels'].cpu().numpy())
            correct_val += (predictions == batch['labels']).sum().item()
            total_val += batch['labels'].size(0)
    average_val_loss = val_loss / len(val_loader)
    accuracy = correct_val / total_val
    return average_val_loss, accuracy, all_predictions, all_true_labels

def objective(trial):
    # Suggest hyperparameters
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
    num_trainable_layers = trial.suggest_int('num_trainable_layers', 1, 6)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])

    # Model setup and modification
    model_config = DistilBertConfig.from_pretrained('distilbert-base-uncased', num_labels=8)
    model = DistilBertForSequenceClassification(model_config)
    model = modify_model(model, num_trainable_layers, dropout_rate)
    model.to(device)

    # Optimizer and scheduler
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

    # Training and validation
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    train_losses = []
    val_losses = []
    for epoch in range(2):  # Number of epochs could be part of the trial hyperparameters
        train_loss, train_accuracy = train_one_epoch(model, train_loader, optimizer, device)
        val_loss, val_accuracy, all_predictions, all_true_labels = validate_model(model, val_loader, device)
        scheduler.step()
        train_losses.append(train_loss)
        val_losses.append(val_loss)

    # Use any metric or combination for optimization
    return val_losses[-1]  # Optimize on the last validation loss


In [ ]:
def modify_model(model, num_trainable_layers, dropout_rate):
    # Freeze layers: only the last 'num_trainable_layers' are trainable
    total_layers = len(model.distilbert.transformer.layer)
    for layer_index, layer in enumerate(model.distilbert.transformer.layer):
        if layer_index < total_layers - num_trainable_layers:
            for param in layer.parameters():
                param.requires_grad = False

    # Adjust dropout rates in applicable transformer layers
    for layer in model.distilbert.transformer.layer:
        layer.attention.dropout.p = dropout_rate
        layer.ffn.dropout.p = dropout_rate

    return model

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)  # Adjust the number of trials as needed

print("Best trial:")
print(study.best_trial.params)